# Prerrequisites

Installing Spark

---



In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip -q install findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

Starting Spark Session and print the version


---


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.2.0'

Creating tunnel</br>
**To Check the Spark UI, open the URL printed by running the above command : https://######/jobs/, /SQL/**


In [5]:
 from google.colab.output import eval_js
 print(eval_js("google.colab.kernel.proxyPort(4040)") + "jobs/")

https://xbig78x3ld-496ff2e9c6d22116-4040-colab.googleusercontent.com/jobs/


# Download Datasets

In [6]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/netflix_titles.csv -P /dataset
!ls /dataset

bank.csv  characters.csv  netflix_titles.csv  vehicles.csv


# Reading Data with Spark SQL

---



## Example 1

In [7]:
!head /dataset/bank.csv

"age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"
30;"unemployed";"married";"primary";"no";1787;"no";"no";"cellular";19;"oct";79;1;-1;0;"unknown";"no"
33;"services";"married";"secondary";"no";4789;"yes";"yes";"cellular";11;"may";220;1;339;4;"failure";"no"
35;"management";"single";"tertiary";"no";1350;"yes";"no";"cellular";16;"apr";185;1;330;1;"failure";"no"
30;"management";"married";"tertiary";"no";1476;"yes";"yes";"unknown";3;"jun";199;4;-1;0;"unknown";"no"
59;"blue-collar";"married";"secondary";"no";0;"yes";"no";"unknown";5;"may";226;1;-1;0;"unknown";"no"
35;"management";"single";"tertiary";"no";747;"no";"no";"cellular";23;"feb";141;2;176;3;"failure";"no"
36;"self-employed";"married";"tertiary";"no";307;"yes";"no";"cellular";14;"may";341;1;330;2;"other";"no"
39;"technician";"married";"secondary";"no";147;"yes";"no";"cellular";6;"may";151;2;-1;0;"unknown";"no"
41;"entrepreneur";"marrie

Converting a RDD to a DataFrame

In [8]:
from pyspark.sql.types import Row
from pyspark.sql.functions import *

bankText = spark.sparkContext.textFile("/dataset/bank.csv")

bank = bankText.map(lambda lineaCsv: lineaCsv.split(";"))\
.filter(lambda s: s[0] != "\"age\"") \
.map(lambda row: Row(int(row[0]), row[1].replace("\"", ""), row[2].replace("\"", ""), row[3].replace("\"", ""), row[5].replace("\"", ""))) \
.toDF(["age", "job", "marital", "education", "balance"]) \
.withColumn("age", col("age").cast("int"))

In [10]:
bank.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- balance: string (nullable = true)



In [20]:
bank.createOrReplaceTempView("bank")

AttributeError: ignored

Loading a **Google Colab extension** to show a table with filters

In [12]:
%load_ext google.colab.data_table

In [13]:

from pyspark.sql.functions import *

bank_grouped = bank\
.groupBy(bank.marital) \
.agg({"balance": "avg"}) \
.select("marital", col("avg(balance)").alias("balance_avg")) \
.orderBy(col("balance_avg").desc())\

bank_grouped.show()


+--------+------------------+
| marital|       balance_avg|
+--------+------------------+
| married| 1463.195566678584|
|  single|1460.4147157190635|
|divorced|1122.3901515151515|
+--------+------------------+



In [14]:
bank_grouped.toPandas()

,marital,balance_avg
0,married,1463.195567
1,single,1460.414716
2,divorced,1122.390152


In [15]:
spark.sql("SELECT marital, avg(balance) as balance_avg FROM bank group by marital").show()

+--------+------------------+
| marital|       balance_avg|
+--------+------------------+
|divorced|1122.3901515151515|
| married| 1463.195566678584|
|  single|1460.4147157190635|
+--------+------------------+



In [16]:
import plotly.express as px

fig = px.pie(bank_grouped.toPandas(), values='balance_avg', names='marital', title='By Marital')
fig.show()

## Example 2

Loading a CSV file as RDD, converting into a DataFrame, applying a specific schema using the method `createDataFrame`

In [21]:
from pyspark.sql.types import *

bankSchema = StructType([
    StructField("age", IntegerType(), False), 
    StructField("job", StringType(), False),
    StructField("marital", StringType(), False),
    StructField("education", StringType(), False),
    StructField("balance", IntegerType(), False)])

bankText = spark.sparkContext.textFile("/dataset/bank.csv")

bank = bankText\
.map(lambda s: s.split(";")).filter(lambda s: s[0] != "\"age\"")\
.map(lambda s:(int(s[0]), str(s[1]).replace("\"", ""), str(s[2]).replace("\"", ""), str(s[3]).replace("\"", ""), int(s[5]) ))

bankdf = spark.createDataFrame(bank, bankSchema)
bankdf.createOrReplaceTempView("bank2")

In [22]:
spark.sql("select * from bank2 limit 10").show()

+---+-------------+-------+---------+-------+
|age|          job|marital|education|balance|
+---+-------------+-------+---------+-------+
| 30|   unemployed|married|  primary|   1787|
| 33|     services|married|secondary|   4789|
| 35|   management| single| tertiary|   1350|
| 30|   management|married| tertiary|   1476|
| 59|  blue-collar|married|secondary|      0|
| 35|   management| single| tertiary|    747|
| 36|self-employed|married| tertiary|    307|
| 39|   technician|married|secondary|    147|
| 41| entrepreneur|married| tertiary|    221|
| 43|     services|married|  primary|    -88|
+---+-------------+-------+---------+-------+



## Exercise 1
Load file `vehicles.csv` to a DataFrame, showing the content and printing the schema.

Use this [documentation](https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html) to read data in a DataFrame

---



In [23]:
!head /dataset/vehicles.csv

name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class
Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled
T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft
X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,NA,repulsorcraft
TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,NA,6.4,1200,1,0,65,2 days,starfighter
Snowspeeder,t-47 airspeeder,Incom corporation,NA,4.5,650,2,0,10,none,airspeeder
TIE bomber,TIE/sa bomber,Sienar Fleet Systems,NA,7.8,850,1,0,none,2 days,space/planetary bomber
AT-AT,All Terrain Armored Transport,"Kuat Drive Yards, Imperial Department of Military Research",NA,20,60,5,40,1000,NA,assault walker
AT-ST,All Terrain Scout Transport,"Kuat Drive Yards, Imperial Department of Military Research",NA,2,90,2,0,200,none,walker
Storm IV Twin-Pod cloud car,Storm IV Twin-Pod,Bespin

In [24]:
vehiclesDF_all = spark.read.format("csv")\
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/vehicles.csv")


In [25]:
vehiclesDF_all.show(truncate=False)

+------------------------------+-------------------------------+----------------------------------------------------------+---------------+------+----------------------+----+----------+--------------+---------------+-------------------------+
|name                          |model                          |manufacturer                                              |cost_in_credits|length|max_atmosphering_speed|crew|passengers|cargo_capacity|consumables    |vehicle_class            |
+------------------------------+-------------------------------+----------------------------------------------------------+---------------+------+----------------------+----+----------+--------------+---------------+-------------------------+
|Sand Crawler                  |Digger Crawler                 |Corellia Mining Corporation                               |150000         |36.8  |30                    |46  |30        |50000         |2 months       |wheeled                  |
|T-16 skyhopper             

In [26]:
vehiclesDF_all.printSchema()

root
 |-- name: string (nullable = true)
 |-- model: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- cost_in_credits: string (nullable = true)
 |-- length: string (nullable = true)
 |-- max_atmosphering_speed: string (nullable = true)
 |-- crew: integer (nullable = true)
 |-- passengers: string (nullable = true)
 |-- cargo_capacity: string (nullable = true)
 |-- consumables: string (nullable = true)
 |-- vehicle_class: string (nullable = true)



Filter out the previous DafaFrame to get vehicles where the capicity is greater than 70

---



In [27]:
vehiclesDF_all \
  .withColumn("cargo_capacity", col("cargo_capacity").cast("float")) \
  .filter(col("cargo_capacity") > 70) \
  .show()

+--------------------+--------------------+--------------------+---------------+------+----------------------+----+----------+--------------+---------------+--------------------+
|                name|               model|        manufacturer|cost_in_credits|length|max_atmosphering_speed|crew|passengers|cargo_capacity|    consumables|       vehicle_class|
+--------------------+--------------------+--------------------+---------------+------+----------------------+----+----------+--------------+---------------+--------------------+
|        Sand Crawler|      Digger Crawler|Corellia Mining C...|         150000|  36.8|                    30|  46|        30|       50000.0|       2 months|             wheeled|
|               AT-AT|All Terrain Armor...|Kuat Drive Yards,...|             NA|    20|                    60|   5|        40|        1000.0|             NA|      assault walker|
|               AT-ST|All Terrain Scout...|Kuat Drive Yards,...|             NA|     2|                  

# Spark SQL. Aggregation Functions

Useful Links:

http://spark.apache.org/docs/latest/api/python/


## Exercise 2

Using the DataFrame with all vehicles loaded in Exercise 1, get the number of passengers by vehicle class


---




In [28]:
from pyspark.sql.types import IntegerType

vehiclesDF_all \
  .withColumn("passengers", col("passengers").cast(IntegerType())) \
  .groupBy("vehicle_class") \
  .avg("passengers") \
  .show()

+--------------------+------------------+
|       vehicle_class|   avg(passengers)|
+--------------------+------------------+
|          droid tank|               4.0|
|space/planetary b...|               0.0|
|   droid starfighter|               0.0|
|      wheeled walker|             150.5|
|          sail barge|             500.0|
|       landing craft|             284.0|
|             speeder|0.3333333333333333|
|      assault walker|              40.0|
|         starfighter|               0.0|
|         air speeder|               0.0|
|           transport|               6.0|
|repulsorcraft car...|              16.0|
|             wheeled|              30.0|
|          airspeeder|               0.8|
|           submarine|               2.0|
|             gunship|              15.0|
|       repulsorcraft|17.285714285714285|
|fire suppression ...|              null|
|              walker|              16.5|
+--------------------+------------------+



## Exercise 3

Load the file `characters.csv` getting the most common eye color among all characters

---

In [29]:
characters_df = spark.read.format("csv") \
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/characters.csv") \
  
characters_df.toPandas()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,Human
1,C-3PO,167,75,NA,gold,yellow,112BBY,NA,Tatooine,Droid
2,R2-D2,96,32,NA,"white, blue",red,33BBY,NA,Naboo,Droid
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,Human
4,Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,Human
...,...,...,...,...,...,...,...,...,...,...
82,Rey,NA,NA,brown,light,hazel,NA,female,NA,Human
83,Poe Dameron,NA,NA,brown,light,brown,NA,male,NA,Human
84,BB8,NA,NA,none,none,black,NA,none,NA,Droid
85,Captain Phasma,NA,NA,NA,NA,NA,NA,female,NA,NA


In [30]:
characters_df \
  .groupBy("eye_color") \
  .count() \
  .orderBy(col("count").desc()) \
  .limit(1) \
  .show() \

+---------+-----+
|eye_color|count|
+---------+-----+
|    brown|   21|
+---------+-----+



## Exercise 4

1. Load characters DataFrame into a temporary table
2. Using SQL, get the number of characters by gender


---



In [31]:
characters_df \
.withColumn("gender",
    when(col("gender") == "none", lit("NA")) \
        .otherwise(characters_df.gender)) \
.registerTempTable("characters")

/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/dataframe.py:140: FutureWarning:

Deprecated in 2.0, use createOrReplaceTempView instead.



In [32]:
byGender = spark.sql("select gender, count(*) as count from characters group by gender")
byGender.show()

+-------------+-----+
|       gender|count|
+-------------+-----+
|           NA|    5|
|hermaphrodite|    1|
|       female|   19|
|         male|   62|
+-------------+-----+



In [33]:
import plotly.express as px

fig = px.pie(byGender.toPandas(), values='count', names='gender', title='By Gender')
fig.show()

## Exercise 5

Load `netflix_titles.csv` file in a DataFrame, printing the schema

---



In [34]:
!head /dataset/netflix_titles.csv

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole Howard, Jennifer Cameron, Jonathan Holmes, Lee Tockar, Lisa Durupt, Maya Kay, Michael Dobson","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies","Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first."
80117401,Movie,Jandino: Whatever it Takes,,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,"Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of ""Sex on Fire"" in his comedy show."
70234439,TV Show,Transformers Prime,,"Peter Cullen, Sumalee Montano, Frank Welker, Jeffrey Combs, Kevin Michael Richardson, Tania Gun

In [35]:
netflixDF_all = spark.read.format("csv")\
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/netflix_titles.csv")


In [36]:
netflixDF_all.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [37]:
netflixDF_all.show()

+--------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+--------+--------+--------------------+--------------------+
| show_id|   type|               title|            director|                cast|             country|       date_added|release_year|  rating|duration|           listed_in|         description|
+--------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+--------+--------+--------------------+--------------------+
|81145628|  Movie|Norm of the North...|Richard Finn, Tim...|Alan Marriott, An...|United States, In...|September 9, 2019|        2019|   TV-PG|  90 min|Children & Family...|Before planning a...|
|80117401|  Movie|Jandino: Whatever...|                null|    Jandino Asporaat|      United Kingdom|September 9, 2016|        2016|   TV-MA|  94 min|     Stand-Up Comedy|"Jandino Asporaat...|
|70234439|TV Show|  Transforme

## Exercise 6

Get the year in which most films were added(No TV Shows). Use a UDF to get the year

---



In [38]:
from pyspark.sql.functions import *

netflixDF_movies = netflixDF_all\
  .filter(netflixDF_all.type == "Movie") \

netflixDF_movies.show()

+--------+-----+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+
| show_id| type|               title|            director|                cast|             country|       date_added|release_year|rating|duration|           listed_in|         description|
+--------+-----+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+
|81145628|Movie|Norm of the North...|Richard Finn, Tim...|Alan Marriott, An...|United States, In...|September 9, 2019|        2019| TV-PG|  90 min|Children & Family...|Before planning a...|
|80117401|Movie|Jandino: Whatever...|                null|    Jandino Asporaat|      United Kingdom|September 9, 2016|        2016| TV-MA|  94 min|     Stand-Up Comedy|"Jandino Asporaat...|
|80125979|Movie|        #realityhigh|    Fernando 

In [39]:
from pyspark.sql.functions import *

def getYearAdded(s): return s[-4:]
getYearAdded_udf = udf(getYearAdded)

netflixDF_movies \
  .filter(col("date_added").isNotNull()) \
  .select(col("title"), getYearAdded_udf("date_added").alias("release_year")) \
  .distinct() \
  .groupBy(col("release_year")) \
  .count() \
  .orderBy(col("count").desc()) \
  .first()

Row(release_year='2019', count=1540)